# Data cleaning and aggregating

## Libraries

In [1]:
import pandas as pd
import numpy as np

## Load data

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.multi_sparse', False)

In [3]:
# Global variables
RESOURCE_FILES = '../data/conjunto_de_datos_recursos_esep_2024'

egresos = pd.read_csv('../data/egresos_hospitalarios_issste2024.csv', sep=';')
clues = pd.read_excel('../data/ESTABLECIMIENTO_SALUD_202508.xlsx', sheet_name='CLUES_202508')
resources = pd.read_csv(f'{RESOURCE_FILES}/conjunto_de_datos/conjunto_de_datos_recursos_esep_2024.csv')
place_type = pd.read_csv(f'{RESOURCE_FILES}/catalogos/tc_esep_tipo_establecimiento.csv')
size_locality = pd.read_csv(f'{RESOURCE_FILES}/catalogos/tc_esep_tamaño de localidad.csv')
ageb = pd.read_csv(f'{RESOURCE_FILES}/catalogos/tc_esep_ageb.csv')
lat_lon_places = pd.read_csv('../data/AGEEML_20259251148648_utf.csv',low_memory=False)

## Explora data

In [4]:
egresos.head(2)

,entidad,unidad,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010
0,Ciudad de Mexico,Cmn 20 de Noviembre,DFIST000312,67,hombre,Cirugia General,pensionado,14/01/2024 0:00,16/01/2024 0:00,N40.X,Hiperplasia de la prostata
1,Guanajuato,Ch Guanajuato Gto.,GTIST000074,45,mujer,Medicina Interna,trabajadora,10/01/2024 0:00,23/01/2024 0:00,K92.2,Hemorragia gastrointestinal no especificada


In [5]:
clues.head(2)

,CLUES,CLAVE DE LA INSTITUCION,NOMBRE DE LA INSTITUCION,CLAVE DE LA ENTIDAD,ENTIDAD,CLAVE DEL MUNICIPIO,MUNICIPIO,CLAVE DE LA LOCALIDAD,LOCALIDAD,CLAVE DE LA JURISDICCION,JURISDICCION,CLAVE DEL TIPO ESTABLECIMIENTO,NOMBRE TIPO ESTABLECIMIENTO,CLAVE DE TIPOLOGIA,NOMBRE DE TIPOLOGIA,CLAVE DE SUBTIPOLOGIA,NOMBRE DE SUBTIPOLOGIA,NOMBRE DE LA UNIDAD,NOMBRE COMERCIAL,CLAVE TIPO DE VIALIDAD,TIPO DE VIALIDAD,VIALIDAD,NUMERO EXTERIOR,NUMERO INTERIOR,CLAVE TIPO DE ASENTAMIENTO,TIPO DE ASENTAMIENTO,ASENTAMIENTO,CODIGO POSTAL,OBSERVACIONES DE LA DIRECCION,CLAVE ESTATUS DE OPERACION,ESTATUS DE OPERACION,RFC DEL ESTABLECIMIENTO,TELEFONO 1 DEL ESTABLECIMIENTO,EXTENSION TELEFONICA 1 DEL ESTABLECIMIENTO,TELEFONO 2 DEL ESTABLECIMIENTO,EXTENSION TELEFONICA 2 DEL ESTABLECIMIENTO,FECHA DE CONSTRUCCION,FECHA DE INICIO DE OPERACION,CLAVE UNIDAD MOVIL MARCA,UNIDAD MOVIL MARCA,UNIDAD MOVIL MARCA ESPECIFICA,UNIDAD MOVIL MODELO,CLAVE UNIDAD MOVIL PROGRAMA,UNIDAD MOVIL PROGRAMA,CLAVE UNIDAD MOVIL TIPO,UNIDAD MOVIL TIPO,CLAVE UNIDAD MOVIL TIPOLOGIA,UNIDAD MOVIL TIPOLOGIA,CLAVE DE LA INS ADM,NOMBRE DE LA INS ADM,CLAVE NIVEL ATENCION,NIVEL ATENCION,CLAVE ESTRATO UNIDAD,ESTRATO UNIDAD,CLAVE TIPO OBRA,TIPO OBRA,CLAVE PROPIEDAD DEL INMUEBLE,PROPIEDAD DEL INMUEBLE,LATITUD,LONGITUD,CLAVE ULTIMO MOVIMIENTO,ULTIMO MOVIMIENTO,FECHA ULTIMO MOVIMIENTO,COMENTARIOS DE LA VALIDACION,CLAVE MOTIVO BAJA,MOTIVO BAJA,FECHA EFECTIVA DE BAJA
0,ASCIJ000012,CIJ,CENTROS DE INTEGRACION JUVENIL,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,1,DE CONSULTA EXTERNA,CPA,CENTRO DE PREVENCION EN ADICCIONES,99,NO ESPECIFICADO,"CENTROS DE INTEGRACIÓN JUVENIL, A.C. UNIDAD OP...",CIJ AGUASCALIENTES,5.0,CALLE,EMILIANO ZAPATA,117,NaN,7.0,COLONIA,AGUASCALIENTES CENTRO,20000.0,CERCA DE LA PENSIÓN DEL VILLAR,1,EN OPERACION,CIJ731003QK3,4499154542,12,NaN,NaN,01/03/1979,16/05/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIJ,CENTROS DE INTEGRACION JUVENIL,1,PRIMER NIVEL,2,URBANO,6.0,SIN MODIFICACION,2.0,RENTADO,21.882127,-102.300272,A,ALTA,03/10/2024,APROBADA CONFORME A INFORMACIÓN REGISTRADA Y O...,NaN,NaN,NaN
1,ASCRO000012,CRO,CRUZ ROJA MEXICANA,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,1,DE CONSULTA EXTERNA,99,NO ESPECIFICADO,99,NO ESPECIFICADO,CONSULTORIO MEDICO AGUASCALIENTES BASE NORTE,CRUZ ROJA MEXICANA IAP,3.0,AVENIDA,INDEPENDENCIA,1602,NaN,8.0,CONDOMINIO,RESIDENCIAL PULGAS PANDAS NORTE,20138.0,A ESPALDAS DE SUCURSAL BANCARIA BANAMEX,1,EN OPERACION,CRM6702109K6,4499152055,NaN,NaN,NaN,01/01/2024,17/06/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRUZ ROJA,CRUZ ROJA MEXICANA,1,PRIMER NIVEL,2,URBANO,4.0,REHABILITACION,2.0,RENTADO,21.915420,-102.297750,A,ALTA,30/01/2025,APROBADA CONFORME A DOCUMENTOS ENVIADOS POR LA...,NaN,NaN,NaN


In [6]:
lat_lon_places.head(2)

,CVEGEO,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL DE VIVIENDAS HABITADAS
0,10010001,NaN,01,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.879822,-102.296046,1878,F13D19,863893,419168,444725,246259
1,10010094,NaN,01,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",21.871874,-102.373530,1901,F13D18,5,*,*,2


In [7]:
resources.head(2)

,sreanio,sreentidad,sremunic,sreconsec,sretamloc,sretipoest,sreageb,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445
0,2024,1,1,1,15,21,0888,1,168,0,168,0,0,0,168,0,100,0,5,0,35,0,5,0,0,0,23,0,0,0,0,0,0,1,0,52,0,0,25,0,24,0,0,0,1,11,16,0,0,0,15,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,5,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0
1,2024,1,1,2,15,21,0835,0,15,0,15,0,0,0,15,0,0,0,0,0,10,0,0,0,2,0,3,0,0,0,0,0,0,0,0,13,0,0,3,0,0,3,0,0,0,3,7,0,0,0,12,2,9,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
place_type.head(2)

,CVE,DESCRIP
0,21,Hospital general
1,22,Hospital de gineco-obstetricia


In [9]:
size_locality.head(2)

,CVE,DESCRIP
0,1,1.. 999
1,2,1 000 ..1 999


In [10]:
ageb.head(2)

,CLAVE_ENTIDAD,ENTIDAD,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_LOCALIDAD,LOCALIDAD,AGEB
0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,290
1,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,483


## Link AGEB with Latitude & Longitud

### Clean Lat & Lon File

In [11]:
lat_lon_places.CVE_ENT.unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', 'CD',
       'CO', 'CQ', 'CT', 'CY', 'QY'], dtype=object)

In [12]:
lat_lon_places = lat_lon_places[~lat_lon_places['CVE_ENT'].isin(['CD','CO','CQ','CT','CY','QY'])]
lat_lon_places['CVE_ENT'] = lat_lon_places['CVE_ENT'].astype(int)
lat_lon_places['CVE_MUN'] = lat_lon_places['CVE_MUN'].astype(int)
lat_lon_places['CVE_LOC'] = lat_lon_places['CVE_LOC'].astype(int)
lat_lon_places.head(2)

,CVEGEO,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL DE VIVIENDAS HABITADAS
0,10010001,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.879822,-102.296046,1878,F13D19,863893,419168,444725,246259
1,10010094,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",21.871874,-102.373530,1901,F13D18,5,*,*,2


### Join Lat-Lon & AGEB

In [13]:
ageb_lat_lon = pd.merge(
    lat_lon_places,
    ageb,
    left_on=['CVE_ENT','CVE_MUN','CVE_LOC'],
    right_on=['CLAVE_ENTIDAD','CLAVE_MUNICIPIO','CLAVE_LOCALIDAD'],
    how='right'
)
ageb_lat_lon.head(1)

,CVEGEO,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL DE VIVIENDAS HABITADAS,CLAVE_ENTIDAD,ENTIDAD,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_LOCALIDAD,LOCALIDAD,AGEB
0,10010001,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.879822,-102.296046,1878,F13D19,863893,419168,444725,246259,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,290


### Clean Lat-Lon-AGEB File

In [14]:
ageb_lat_lon_cols = [
    'CLAVE_ENTIDAD','CLAVE_MUNICIPIO','CLAVE_LOCALIDAD','ENTIDAD',
    'MUNICIPIO','LOCALIDAD','AGEB','LAT_DECIMAL','LON_DECIMAL','POB_TOTAL']
ageb_lat_lon_cl = ageb_lat_lon[ageb_lat_lon_cols].copy()
ageb_lat_lon_cl.head(2)

,CLAVE_ENTIDAD,CLAVE_MUNICIPIO,CLAVE_LOCALIDAD,ENTIDAD,MUNICIPIO,LOCALIDAD,AGEB,LAT_DECIMAL,LON_DECIMAL,POB_TOTAL
0,1,1,1,Aguascalientes,Aguascalientes,Aguascalientes,290,21.879822,-102.296046,863893
1,1,1,1,Aguascalientes,Aguascalientes,Aguascalientes,483,21.879822,-102.296046,863893


## Clean & Merge Resources with AGEB-LAT-LON

In [15]:
resources.columns, ageb_lat_lon_cl.columns

(Index(['sreanio', 'sreentidad', 'sremunic', 'sreconsec', 'sretamloc',
        'sretipoest', 'sreageb', 'sre378_nh', 'sre378_ae', 'sre379_nh',
        'sre379_ae', 'sre380_nh', 'sre380_ae', 'sre381_nh', 'sre381_ae',
        'sre382_nh', 'sre382_ae', 'sre383_nh', 'sre383_ae', 'sre384_nh',
        'sre384_ae', 'sre385_nh', 'sre385_ae', 'sre386_nh', 'sre386_ae',
        'sre387_nh', 'sre387_ae', 'sre388_nh', 'sre388_ae', 'sre389_nh',
        'sre389_ae', 'sre390_nh', 'sre390_ae', 'sre391_nh', 'sre391_ae',
        'sre392', 'sre393', 'sre394', 'sre395', 'sre396', 'sre397', 'sre398',
        'sre399', 'sre400', 'sre401', 'sre402', 'sre403', 'sre404', 'sre405',
        'sre406', 'sre407', 'sre408', 'sre409', 'sre410', 'sre411', 'sre412',
        'sre413', 'sre414', 'sre415', 'sre416', 'sre417', 'sre418', 'sre419',
        'sre420', 'sre421', 'sre422', 'sre423', 'sre424', 'sre425', 'sre426',
        'sre427', 'sre428', 'sre429', 'sre430', 'sre431', 'sre432', 'sre433',
        'sre434', 'sre43

### Clean resources & AGEB

In [16]:
resources['sreageb'] = resources['sreageb'].str.lstrip('0')
ageb_lat_lon_cl['AGEB'] = ageb_lat_lon_cl['AGEB'].str.lstrip('0')

### Merge hints
- `sreageb`: `AGEB`

In [17]:
resources_cl = pd.merge(
    resources,
    ageb_lat_lon_cl,
    left_on=['sreentidad','sremunic','sreageb'],
    right_on=['CLAVE_ENTIDAD','CLAVE_MUNICIPIO','AGEB'],
    how='left'
)
resources_cl.head(2)

,sreanio,sreentidad,sremunic,sreconsec,sretamloc,sretipoest,sreageb,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445,CLAVE_ENTIDAD,CLAVE_MUNICIPIO,CLAVE_LOCALIDAD,ENTIDAD,MUNICIPIO,LOCALIDAD,AGEB,LAT_DECIMAL,LON_DECIMAL,POB_TOTAL
0,2024,1,1,1,15,21,888,1,168,0,168,0,0,0,168,0,100,0,5,0,35,0,5,0,0,0,23,0,0,0,0,0,0,1,0,52,0,0,25,0,24,0,0,0,1,11,16,0,0,0,15,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,5,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,888,21.879822,-102.296046,863893
1,2024,1,1,2,15,21,835,0,15,0,15,0,0,0,15,0,0,0,0,0,10,0,0,0,2,0,3,0,0,0,0,0,0,0,0,13,0,0,3,0,0,3,0,0,0,3,7,0,0,0,12,2,9,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,835,21.879822,-102.296046,863893


In [18]:
resources_cl.isna().sum(),len(resources_cl)

(sreanio         0
 sreentidad      0
 sremunic        0
 sreconsec       0
 sretamloc       0
                ..
 LOCALIDAD      43
 AGEB           43
 LAT_DECIMAL    43
 LON_DECIMAL    43
 POB_TOTAL      43
 Length: 99, dtype: int64,
 2390)

In [19]:
# Drop nan (few nan values)
resources_cl.dropna(inplace=True)
resources_cl.isna().sum()

sreanio        0
sreentidad     0
sremunic       0
sreconsec      0
sretamloc      0
              ..
LOCALIDAD      0
AGEB           0
LAT_DECIMAL    0
LON_DECIMAL    0
POB_TOTAL      0
Length: 99, dtype: int64

In [20]:
resources_cl.duplicated().sum()

np.int64(0)

In [21]:
resources_cl['LAT_DECIMAL'].nunique(), resources_cl['LON_DECIMAL'].nunique()

(327, 327)

Se identifican latitudes y longitudes repetidas para varios códigos `AGEB`, lo cual indica que puede ser que hacen referencia a la misma ubicación pero son diferentes establecimientos, sin embargo, se busca algo mas general por lo que se agregaran sus datos

In [22]:
resources_cl.head(5)

,sreanio,sreentidad,sremunic,sreconsec,sretamloc,sretipoest,sreageb,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445,CLAVE_ENTIDAD,CLAVE_MUNICIPIO,CLAVE_LOCALIDAD,ENTIDAD,MUNICIPIO,LOCALIDAD,AGEB,LAT_DECIMAL,LON_DECIMAL,POB_TOTAL
0,2024,1,1,1,15,21,888,1,168,0,168,0,0,0,168,0,100,0,5,0,35,0,5,0,0,0,23,0,0,0,0,0,0,1,0,52,0,0,25,0,24,0,0,0,1,11,16,0,0,0,15,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,5,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,888,21.879822,-102.296046,863893
1,2024,1,1,2,15,21,835,0,15,0,15,0,0,0,15,0,0,0,0,0,10,0,0,0,2,0,3,0,0,0,0,0,0,0,0,13,0,0,3,0,0,3,0,0,0,3,7,0,0,0,12,2,9,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,835,21.879822,-102.296046,863893
2,2024,1,1,24,15,26,642,2,0,2,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,12,8,0,0,4,0,0,1,0,2,2,0,10,0,0,0,0,10,0,0,0,0,1,0,0,1,0,0,2,1,2,8,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,642,21.879822,-102.296046,863893
3,2024,1,1,25,15,21,572,5,0,5,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,163,5,2,93,56,34,0,0,0,3,28,35,3,2,1,33,6,12,0,0,15,6,0,0,6,1,0,1,3,0,0,6,2,7,15,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,572,21.879822,-102.296046,863893
4,2024,1,1,28,15,21,483,3,0,3,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,1,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,3,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,483,21.879822,-102.296046,863893


In [23]:
no_choosen_cols_res = ['sreconsec','sretamloc','sretipoest','sreageb','sreentidad','sremunic','AGEB']
resources_cl_unique = resources_cl.drop(columns=no_choosen_cols_res)

In [24]:
resources_cl_unique.shape

(2347, 92)

In [25]:
resources_cl_agg = resources_cl_unique.groupby([
    'sreanio','CLAVE_ENTIDAD','CLAVE_MUNICIPIO','CLAVE_LOCALIDAD','ENTIDAD','MUNICIPIO','LOCALIDAD',
    'LAT_DECIMAL','LON_DECIMAL','POB_TOTAL'
    ]).sum()

In [26]:
resources_cl_agg.reset_index(inplace=True)

In [27]:
resources_cl_agg.head(1)

,sreanio,CLAVE_ENTIDAD,CLAVE_MUNICIPIO,CLAVE_LOCALIDAD,ENTIDAD,MUNICIPIO,LOCALIDAD,LAT_DECIMAL,LON_DECIMAL,POB_TOTAL,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445
0,2024,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,21.879822,-102.296046,863893,108,520,106,517,29,25,61,491,11,159,13,33,5,127,4,21,5,25,23,126,2,0,1,0,13,1,2,3,1294,76,24,617,114,299,61,20,73,50,297,280,79,22,57,399,32,72,50,41,204,138,28,6,104,12,3,15,30,0,0,54,17,64,124,8,20,14,2,1,2,3,13,20,9,0,0,6,0,2,37,18,19,2


## Clean egresos and clues

In [28]:
# Filtrar los CLUES de egresos_cl que no están en clues_cl
egresos[~egresos['clues'].isin(clues['CLUES'])]['clues'].unique()

array(['PENDIENTE'], dtype=object)

In [29]:
len(egresos[egresos['clues']=='PENDIENTE'])

11433

In [30]:
# Filtrar los CLUES de egresos_cl que no están en clues_cl
print(egresos[~egresos['clues'].isin(clues['CLUES'])]['clues'].unique())
print(egresos[~egresos['clues'].isin(clues['CLUES'])]['clues'].shape)

['PENDIENTE']
(11433,)


In [31]:
egresos[egresos['clues']=='PENDIENTE'].nunique()

entidad                           1
unidad                            1
clues                             1
edad_anios                      105
sexo                              2
servicio_troncal                  4
tipo_derechohabiente             17
fecha_ingreso                   388
fecha_egreso                    366
diagnostico_principal_cie10    1538
descripcion_cie_010            1538
dtype: int64

In [32]:
temp = clues[clues['ENTIDAD'].str.upper()=='JALISCO'][
    ['CLUES','ENTIDAD','NOMBRE DE LA INSTITUCION','NOMBRE DE LA UNIDAD','LATITUD','LONGITUD']]
for idx, row in temp.iterrows():
    if 'farias' in row['NOMBRE DE LA UNIDAD'].lower():
        print(row)

CLUES                                                             JCIMO000096
ENTIDAD                                                               JALISCO
NOMBRE DE LA INSTITUCION    INSTITUTO MEXICANO DEL SEGURO SOCIAL REGIMEN B...
NOMBRE DE LA UNIDAD                                              GÓMEZ FARIAS
LATITUD                                                               19.7942
LONGITUD                                                             -103.477
Name: 20863, dtype: object
CLUES                                                             JCIST000132
ENTIDAD                                                               JALISCO
NOMBRE DE LA INSTITUCION    INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...
NOMBRE DE LA UNIDAD                   HR/HAE "VALENTÍN GÓMEZ FARIAS", ZAPOPAN
LATITUD                                                             20.718636
LONGITUD                                                          -103.387316
Name: 21024, dtype: object
CLUES     

After a review on [Google Maps](https://www.google.com/maps/place/Hospital+Regional+Dr.+Valent%C3%ADn+G%C3%B3mez+Far%C3%ADas+-+ISSSTE/@20.7185359,-103.3896644,17z/data=!3m2!4b1!5s0x8428afb72f1a1efb:0xd7304b3901a09c35!4m6!3m5!1s0x8428afb0cd4e7b97:0x3d5f29a3b00b2781!8m2!3d20.7185359!4d-103.3870895!16s%2Fg%2F1tf5f1hj?entry=ttu&g_ep=EgoyMDI1MTAwMS4wIKXMDSoASAFQAw%3D%3D) searching for the place, we deduced that the latitude and longitude is `20.71,-103.38` respectively, so we're going to replace `PENDIENTE` with `JCIST000132` CLUES code.

In [33]:
egresos.replace({'clues':{'PENDIENTE':'JCIST000132'}}, inplace=True)


In [34]:
clues_sel_cols = [
    'CLUES','CLAVE DE LA INSTITUCION','NOMBRE DE LA INSTITUCION','CLAVE DE LA ENTIDAD','ENTIDAD',
    'CLAVE DEL MUNICIPIO','MUNICIPIO','CLAVE DE LA LOCALIDAD','LOCALIDAD','NOMBRE DE LA UNIDAD',
    'CODIGO POSTAL','LATITUD','LONGITUD','NIVEL ATENCION']
clues_cl = clues[clues_sel_cols].copy()
clues_cl['CLUES'] = clues_cl['CLUES'].str.strip().str.upper()
clues_cl.head(2)

,CLUES,CLAVE DE LA INSTITUCION,NOMBRE DE LA INSTITUCION,CLAVE DE LA ENTIDAD,ENTIDAD,CLAVE DEL MUNICIPIO,MUNICIPIO,CLAVE DE LA LOCALIDAD,LOCALIDAD,NOMBRE DE LA UNIDAD,CODIGO POSTAL,LATITUD,LONGITUD,NIVEL ATENCION
0,ASCIJ000012,CIJ,CENTROS DE INTEGRACION JUVENIL,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,"CENTROS DE INTEGRACIÓN JUVENIL, A.C. UNIDAD OP...",20000.0,21.882127,-102.300272,PRIMER NIVEL
1,ASCRO000012,CRO,CRUZ ROJA MEXICANA,1,AGUASCALIENTES,1,AGUASCALIENTES,1,AGUASCALIENTES,CONSULTORIO MEDICO AGUASCALIENTES BASE NORTE,20138.0,21.915420,-102.297750,PRIMER NIVEL


In [35]:
egresos_cl = egresos.drop(columns=['entidad','unidad'])
egresos_cl['clues'] = egresos_cl['clues'].str.strip().str.upper()
egresos_cl.head(2)

,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010
0,DFIST000312,67,hombre,Cirugia General,pensionado,14/01/2024 0:00,16/01/2024 0:00,N40.X,Hiperplasia de la prostata
1,GTIST000074,45,mujer,Medicina Interna,trabajadora,10/01/2024 0:00,23/01/2024 0:00,K92.2,Hemorragia gastrointestinal no especificada


In [36]:
egresos_cl.shape, clues_cl.shape

((355574, 9), (58919, 14))

In [37]:
egresos_cl[~egresos_cl['clues'].isin(clues_cl['CLUES'])]['clues']

Series([], Name: clues, dtype: object)

In [38]:
egresos_cl[egresos_cl['clues']=='PENDIENTE']

,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010


### Join egresos with clues

In [39]:
egresos_clues_cl = pd.merge(egresos_cl, clues_cl, left_on='clues',right_on='CLUES',how='left')
egresos_clues_cl.head(2)

,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010,CLUES,CLAVE DE LA INSTITUCION,NOMBRE DE LA INSTITUCION,CLAVE DE LA ENTIDAD,ENTIDAD,CLAVE DEL MUNICIPIO,MUNICIPIO,CLAVE DE LA LOCALIDAD,LOCALIDAD,NOMBRE DE LA UNIDAD,CODIGO POSTAL,LATITUD,LONGITUD,NIVEL ATENCION
0,DFIST000312,67,hombre,Cirugia General,pensionado,14/01/2024 0:00,16/01/2024 0:00,N40.X,Hiperplasia de la prostata,DFIST000312,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,9,CIUDAD DE MEXICO,14,BENITO JUAREZ,1,BENITO JUAREZ,"CMN ""20 DE NOVIEMBRE""",3104.0,19.372721,-99.171163,TERCER NIVEL
1,GTIST000074,45,mujer,Medicina Interna,trabajadora,10/01/2024 0:00,23/01/2024 0:00,K92.2,Hemorragia gastrointestinal no especificada,GTIST000074,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,11,GUANAJUATO,15,GUANAJUATO,1,GUANAJUATO,"CH GUANAJUATO, GTO.",36000.0,21.004524,-101.253467,SEGUNDO NIVEL


In [40]:
len(egresos_clues_cl) - egresos_clues_cl[['CLAVE DE LA ENTIDAD','CLAVE DEL MUNICIPIO','CLAVE DE LA LOCALIDAD','LATITUD','LONGITUD']].duplicated().sum()

np.int64(120)

## Join egresos cleaned and resources cleaned

In [41]:
resources_cl_unique.columns

Index(['sreanio', 'sre378_nh', 'sre378_ae', 'sre379_nh', 'sre379_ae',
       'sre380_nh', 'sre380_ae', 'sre381_nh', 'sre381_ae', 'sre382_nh',
       'sre382_ae', 'sre383_nh', 'sre383_ae', 'sre384_nh', 'sre384_ae',
       'sre385_nh', 'sre385_ae', 'sre386_nh', 'sre386_ae', 'sre387_nh',
       'sre387_ae', 'sre388_nh', 'sre388_ae', 'sre389_nh', 'sre389_ae',
       'sre390_nh', 'sre390_ae', 'sre391_nh', 'sre391_ae', 'sre392', 'sre393',
       'sre394', 'sre395', 'sre396', 'sre397', 'sre398', 'sre399', 'sre400',
       'sre401', 'sre402', 'sre403', 'sre404', 'sre405', 'sre406', 'sre407',
       'sre408', 'sre409', 'sre410', 'sre411', 'sre412', 'sre413', 'sre414',
       'sre415', 'sre416', 'sre417', 'sre418', 'sre419', 'sre420', 'sre421',
       'sre422', 'sre423', 'sre424', 'sre425', 'sre426', 'sre427', 'sre428',
       'sre429', 'sre430', 'sre431', 'sre432', 'sre433', 'sre434', 'sre435',
       'sre436', 'sre437', 'sre438', 'sre439', 'sre440', 'sre441', 'sre442',
       'sre443', 'sre444

## Clean egresos and resources

In [42]:
egresos_clues_cl.head(3)


,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010,CLUES,CLAVE DE LA INSTITUCION,NOMBRE DE LA INSTITUCION,CLAVE DE LA ENTIDAD,ENTIDAD,CLAVE DEL MUNICIPIO,MUNICIPIO,CLAVE DE LA LOCALIDAD,LOCALIDAD,NOMBRE DE LA UNIDAD,CODIGO POSTAL,LATITUD,LONGITUD,NIVEL ATENCION
0,DFIST000312,67,hombre,Cirugia General,pensionado,14/01/2024 0:00,16/01/2024 0:00,N40.X,Hiperplasia de la prostata,DFIST000312,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,9,CIUDAD DE MEXICO,14,BENITO JUAREZ,1,BENITO JUAREZ,"CMN ""20 DE NOVIEMBRE""",3104.0,19.372721,-99.171163,TERCER NIVEL
1,GTIST000074,45,mujer,Medicina Interna,trabajadora,10/01/2024 0:00,23/01/2024 0:00,K92.2,Hemorragia gastrointestinal no especificada,GTIST000074,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,11,GUANAJUATO,15,GUANAJUATO,1,GUANAJUATO,"CH GUANAJUATO, GTO.",36000.0,21.004524,-101.253467,SEGUNDO NIVEL
2,CSIST000251,39,hombre,Medicina Interna,trabajador,9/01/2024 0:00,23/01/2024 0:00,D69.6,Trombocitopenia no especificada,CSIST000251,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,7,CHIAPAS,101,TUXTLA GUTIERREZ,1,TUXTLA GUTIERREZ,"HG ""DR. BELISARIO DOMINGUEZ"", TUXTLA GUTIÉRREZ.",29060.0,16.750797,-93.129676,SEGUNDO NIVEL


In [43]:
resources_cl_agg.head(1)

,sreanio,CLAVE_ENTIDAD,CLAVE_MUNICIPIO,CLAVE_LOCALIDAD,ENTIDAD,MUNICIPIO,LOCALIDAD,LAT_DECIMAL,LON_DECIMAL,POB_TOTAL,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445
0,2024,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,21.879822,-102.296046,863893,108,520,106,517,29,25,61,491,11,159,13,33,5,127,4,21,5,25,23,126,2,0,1,0,13,1,2,3,1294,76,24,617,114,299,61,20,73,50,297,280,79,22,57,399,32,72,50,41,204,138,28,6,104,12,3,15,30,0,0,54,17,64,124,8,20,14,2,1,2,3,13,20,9,0,0,6,0,2,37,18,19,2


In [44]:
def truncar_1_decimal(x):
    return np.trunc(x * 10) / 10

egresos_clues_cl.loc[:, 'lat_trunc'] = truncar_1_decimal(egresos_clues_cl['LATITUD'])
egresos_clues_cl.loc[:, 'lon_trunc'] = truncar_1_decimal(egresos_clues_cl['LONGITUD'])
resources_cl_agg.loc[:, 'lat_trunc_res'] = truncar_1_decimal(resources_cl_agg['LAT_DECIMAL'])
resources_cl_agg.loc[:, 'lon_trunc_res'] = truncar_1_decimal(resources_cl_agg['LON_DECIMAL'])


In [45]:
print(f'{resources_cl_agg[['CLAVE_ENTIDAD','CLAVE_MUNICIPIO','CLAVE_LOCALIDAD','lat_trunc_res','lon_trunc_res']].duplicated().sum()=}')
print(f'{egresos_clues_cl[['CLAVE DE LA ENTIDAD','CLAVE DEL MUNICIPIO','CLAVE DE LA LOCALIDAD','lat_trunc','lon_trunc']].duplicated().sum()=}')

resources_cl_agg[['CLAVE_ENTIDAD','CLAVE_MUNICIPIO','CLAVE_LOCALIDAD','lat_trunc_res','lon_trunc_res']].duplicated().sum()=np.int64(0)
egresos_clues_cl[['CLAVE DE LA ENTIDAD','CLAVE DEL MUNICIPIO','CLAVE DE LA LOCALIDAD','lat_trunc','lon_trunc']].duplicated().sum()=np.int64(355457)


In [46]:
resources_cl_unique.head()

,sreanio,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445,CLAVE_ENTIDAD,CLAVE_MUNICIPIO,CLAVE_LOCALIDAD,ENTIDAD,MUNICIPIO,LOCALIDAD,LAT_DECIMAL,LON_DECIMAL,POB_TOTAL
0,2024,1,168,0,168,0,0,0,168,0,100,0,5,0,35,0,5,0,0,0,23,0,0,0,0,0,0,1,0,52,0,0,25,0,24,0,0,0,1,11,16,0,0,0,15,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,5,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,21.879822,-102.296046,863893
1,2024,0,15,0,15,0,0,0,15,0,0,0,0,0,10,0,0,0,2,0,3,0,0,0,0,0,0,0,0,13,0,0,3,0,0,3,0,0,0,3,7,0,0,0,12,2,9,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,21.879822,-102.296046,863893
2,2024,2,0,2,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,12,8,0,0,4,0,0,1,0,2,2,0,10,0,0,0,0,10,0,0,0,0,1,0,0,1,0,0,2,1,2,8,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,21.879822,-102.296046,863893
3,2024,5,0,5,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,163,5,2,93,56,34,0,0,0,3,28,35,3,2,1,33,6,12,0,0,15,6,0,0,6,1,0,1,3,0,0,6,2,7,15,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,21.879822,-102.296046,863893
4,2024,3,0,3,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,1,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,3,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,Aguascalientes,Aguascalientes,Aguascalientes,21.879822,-102.296046,863893


In [47]:
egresos_clues_cl.head(2)

,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010,CLUES,CLAVE DE LA INSTITUCION,NOMBRE DE LA INSTITUCION,CLAVE DE LA ENTIDAD,ENTIDAD,CLAVE DEL MUNICIPIO,MUNICIPIO,CLAVE DE LA LOCALIDAD,LOCALIDAD,NOMBRE DE LA UNIDAD,CODIGO POSTAL,LATITUD,LONGITUD,NIVEL ATENCION,lat_trunc,lon_trunc
0,DFIST000312,67,hombre,Cirugia General,pensionado,14/01/2024 0:00,16/01/2024 0:00,N40.X,Hiperplasia de la prostata,DFIST000312,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,9,CIUDAD DE MEXICO,14,BENITO JUAREZ,1,BENITO JUAREZ,"CMN ""20 DE NOVIEMBRE""",3104.0,19.372721,-99.171163,TERCER NIVEL,19.3,-99.1
1,GTIST000074,45,mujer,Medicina Interna,trabajadora,10/01/2024 0:00,23/01/2024 0:00,K92.2,Hemorragia gastrointestinal no especificada,GTIST000074,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,11,GUANAJUATO,15,GUANAJUATO,1,GUANAJUATO,"CH GUANAJUATO, GTO.",36000.0,21.004524,-101.253467,SEGUNDO NIVEL,21.0,-101.2


### Merge egresos and resources

In [48]:
egresos_resources = pd.merge(
    resources_cl_agg,
    egresos_clues_cl,
    left_on=['CLAVE_ENTIDAD','CLAVE_MUNICIPIO','CLAVE_LOCALIDAD','lat_trunc_res','lon_trunc_res'],
    right_on=['CLAVE DE LA ENTIDAD','CLAVE DEL MUNICIPIO','CLAVE DE LA LOCALIDAD','lat_trunc','lon_trunc'],
    how='inner'
)
egresos_resources.head(1)

,sreanio,CLAVE_ENTIDAD,CLAVE_MUNICIPIO,CLAVE_LOCALIDAD,ENTIDAD_x,MUNICIPIO_x,LOCALIDAD_x,LAT_DECIMAL,LON_DECIMAL,POB_TOTAL,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445,lat_trunc_res,lon_trunc_res,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010,CLUES,CLAVE DE LA INSTITUCION,NOMBRE DE LA INSTITUCION,CLAVE DE LA ENTIDAD,ENTIDAD_y,CLAVE DEL MUNICIPIO,MUNICIPIO_y,CLAVE DE LA LOCALIDAD,LOCALIDAD_y,NOMBRE DE LA UNIDAD,CODIGO POSTAL,LATITUD,LONGITUD,NIVEL ATENCION,lat_trunc,lon_trunc
0,2024,2.0,2.0,1.0,Baja California,Mexicali,Mexicali,32.641176,-115.475578,854186,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,72,hombre,Cirugia General,pensionado,12/01/2024 0:00,19/01/2024 0:00,C61.X,Tumor maligno de la prostata,BCIST000040,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4


In [49]:
egresos_resources.shape

(243562, 119)

### Clean egresos and resources file

In [50]:
choosen_cols_eg_res = [
    'CLAVE_ENTIDAD','CLAVE_MUNICIPIO','CLAVE_LOCALIDAD','ENTIDAD_x','MUNICIPIO_x','LOCALIDAD_x',
    'POB_TOTAL','CLUES'
]
egresos_resources.drop(columns=choosen_cols_eg_res, inplace=True)
egresos_resources.head()



,sreanio,LAT_DECIMAL,LON_DECIMAL,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445,lat_trunc_res,lon_trunc_res,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010,CLAVE DE LA INSTITUCION,NOMBRE DE LA INSTITUCION,CLAVE DE LA ENTIDAD,ENTIDAD_y,CLAVE DEL MUNICIPIO,MUNICIPIO_y,CLAVE DE LA LOCALIDAD,LOCALIDAD_y,NOMBRE DE LA UNIDAD,CODIGO POSTAL,LATITUD,LONGITUD,NIVEL ATENCION,lat_trunc,lon_trunc
0,2024,32.641176,-115.475578,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,72,hombre,Cirugia General,pensionado,12/01/2024 0:00,19/01/2024 0:00,C61.X,Tumor maligno de la prostata,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4
1,2024,32.641176,-115.475578,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,73,mujer,Cirugia General,madre,15/01/2024 0:00,22/01/2024 0:00,M17.9,Gonartrosis no especificada,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4
2,2024,32.641176,-115.475578,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,81,mujer,Gineco-Obstetricia,pensionada,22/01/2024 0:00,22/01/2024 0:00,N95.0,Hemorragia postmenopausica,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4
3,2024,32.641176,-115.475578,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,82,hombre,Medicina Interna,pensionado,15/02/2024 0:00,16/02/2024 0:00,I50.1,Insuficiencia ventricular izquierda,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4
4,2024,32.641176,-115.475578,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,74,mujer,Cirugia General,pensionada,8/02/2024 0:00,9/02/2024 0:00,C50.9,Tumor maligno de la mama parte no especificada,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4


In [51]:
egresos_clues_cl.shape, resources_cl_unique.shape

((355574, 25), (2347, 92))

In [52]:
egresos_resources.head(1)

,sreanio,LAT_DECIMAL,LON_DECIMAL,sre378_nh,sre378_ae,sre379_nh,sre379_ae,sre380_nh,sre380_ae,sre381_nh,sre381_ae,sre382_nh,sre382_ae,sre383_nh,sre383_ae,sre384_nh,sre384_ae,sre385_nh,sre385_ae,sre386_nh,sre386_ae,sre387_nh,sre387_ae,sre388_nh,sre388_ae,sre389_nh,sre389_ae,sre390_nh,sre390_ae,sre391_nh,sre391_ae,sre392,sre393,sre394,sre395,sre396,sre397,sre398,sre399,sre400,sre401,sre402,sre403,sre404,sre405,sre406,sre407,sre408,sre409,sre410,sre411,sre412,sre413,sre414,sre415,sre416,sre417,sre418,sre419,sre420,sre421,sre422,sre423,sre424,sre425,sre426,sre427,sre428,sre429,sre430,sre431,sre432,sre433,sre434,sre435,sre436,sre437,sre438,sre439,sre440,sre441,sre442,sre443,sre444,sre445,lat_trunc_res,lon_trunc_res,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_cie_010,CLAVE DE LA INSTITUCION,NOMBRE DE LA INSTITUCION,CLAVE DE LA ENTIDAD,ENTIDAD_y,CLAVE DEL MUNICIPIO,MUNICIPIO_y,CLAVE DE LA LOCALIDAD,LOCALIDAD_y,NOMBRE DE LA UNIDAD,CODIGO POSTAL,LATITUD,LONGITUD,NIVEL ATENCION,lat_trunc,lon_trunc
0,2024,32.641176,-115.475578,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,72,hombre,Cirugia General,pensionado,12/01/2024 0:00,19/01/2024 0:00,C61.X,Tumor maligno de la prostata,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4


In [53]:
rename_dict = {
    # Año y localización
    'sreanio': 'anio',
    'sreentidad': 'clave_entidad',
    'sremunic': 'clave_municipio',
    'sreconsec': 'consecutivo',
    'sretamloc': 'tamano_localidad',
    'sretipoest': 'tipo_establecimiento',
    'sreageb': 'ageb',

    # Coordenadas
    'LAT_DECIMAL': 'lat_decimal',
    'LON_DECIMAL': 'lon_decimal',
    'LATITUD': 'latitud',
    'LONGITUD': 'longitud',
    'lat_trunc_res': 'lat_trunc_res',
    'lon_trunc_res': 'lon_trunc_res',
    'lat_trunc': 'lat_trunc',
    'lon_trunc': 'lon_trunc',

    # Datos de egresos
    'clues': 'clues',
    'edad_anios': 'edad_anios',
    'sexo': 'sexo',
    'servicio_troncal': 'servicio_troncal',
    'tipo_derechohabiente': 'tipo_derechohabiente',
    'fecha_ingreso': 'fecha_ingreso',
    'fecha_egreso': 'fecha_egreso',
    'diagnostico_principal_cie10': 'diagnostico_principal_cie10',
    'descripcion_cie_010': 'descripcion_diagnostico',

    # Institución y unidad médica
    'CLAVE DE LA INSTITUCION': 'clave_institucion',
    'NOMBRE DE LA INSTITUCION': 'nombre_institucion',
    'CLAVE DE LA ENTIDAD': 'clave_entidad_egresos',
    'ENTIDAD_y': 'nombre_entidad',
    'CLAVE DEL MUNICIPIO': 'clave_municipio_egresos',
    'MUNICIPIO_y': 'nombre_municipio',
    'CLAVE DE LA LOCALIDAD': 'clave_localidad',
    'LOCALIDAD_y': 'nombre_localidad',
    'NOMBRE DE LA UNIDAD': 'nombre_unidad',
    'CODIGO POSTAL': 'codigo_postal',
    'NIVEL ATENCION': 'nivel_atencion',

    # Recursos humanos y materiales (fragmento, puedes extenderlo más si gustas)
    'sre378_nh': 'personal_medico_nomina',
    'sre378_ae': 'personal_medico_acuerdo',
    'sre379_nh': 'contacto_directo_paciente_nomina',
    'sre379_ae': 'contacto_directo_paciente_acuerdo',
    'sre380_nh': 'medicos_generales_nomina',
    'sre380_ae': 'medicos_generales_acuerdo',
    'sre381_nh': 'especialistas_nomina',
    'sre381_ae': 'especialistas_acuerdo',
    'sre382_nh': 'ginecoobstetras_nomina',
    'sre382_ae': 'ginecoobstetras_acuerdo',
    'sre383_nh': 'pediatras_nomina',
    'sre383_ae': 'pediatras_acuerdo',
    'sre384_nh': 'cirujanos_nomina',
    'sre384_ae': 'cirujanos_acuerdo',
    'sre385_nh': 'internistas_nomina',
    'sre385_ae': 'internistas_acuerdo',
    'sre386_nh': 'anestesiologos_nomina',
    'sre386_ae': 'anestesiologos_acuerdo',
    'sre387_nh': 'otros_especialistas_nomina',
    'sre387_ae': 'otros_especialistas_acuerdo',
    'sre388_nh': 'odontologos_nomina',
    'sre388_ae': 'odontologos_acuerdo',
    'sre389_nh': 'residentes_nomina',
    'sre389_ae': 'residentes_acuerdo',
    'sre390_nh': 'pasantes_nomina',
    'sre390_ae': 'pasantes_acuerdo',
    'sre391_nh': 'medicos_otras_labores_nomina',
    'sre391_ae': 'medicos_otras_labores_acuerdo',
    'sre392': 'personal_no_medico',
    'sre393': 'diagnostico_medico',
    'sre394': 'tratamiento_medico',
    'sre395': 'personal_paramedico',
    'sre396': 'auxiliares_enfermeria',
    'sre397': 'enfermeras_generales',
    'sre398': 'enfermeras_especializadas',
    'sre399': 'pasantes_enfermeria',
    'sre400': 'otras_enfermeras',
    'sre401': 'otro_personal_paramedico',
    'sre402': 'personal_administrativo',
    'sre403': 'otro_personal',
    'sre404': 'consultorios',
    'sre405': 'consultorios_generales',
    'sre406': 'consultorios_especialidad',
    'sre407': 'camas_censables',
    'sre408': 'camas_medicina_interna',
    'sre409': 'camas_cirugia',
    'sre410': 'camas_ginecoobstetricia',
    'sre411': 'camas_pediatria',
    'sre412': 'camas_otras',
    'sre413': 'camas_no_censables',
    'sre414': 'camas_cuidado_intensivo',
    'sre415': 'camas_cuidado_intermedio',
    'sre416': 'camas_no_censables_otras',
    'sre417': 'lab_analisis_clinicos',
    'sre418': 'lab_anatomia_patologica',
    'sre419': 'salas_radiologia',
    'sre420': 'equipos_rayos_x',
    'sre421': 'area_radioterapia',
    'sre422': 'equipos_radioterapia',
    'sre423': 'quirofanos',
    'sre424': 'salas_expulsion',
    'sre425': 'incubadoras',
    'sre426': 'cunas_rn',
    'sre427': 'area_pediatria',
    'sre428': 'area_urgencias',
    'sre429': 'area_aislamiento',
    'sre430': 'resonancia_magnetica',
    'sre431': 'equipo_dialisis',
    'sre432': 'hemodialisis',
    'sre433': 'mamografia',
    'sre434': 'ultrasonido',
    'sre435': 'electrocardiografo',
    'sre436': 'endoscopio',
    'sre437': 'electroencefalografo',
    'sre438': 'litotriptores',
    'sre439': 'tac_scanner',
    'sre440': 'bomba_cobalto',
    'sre441': 'bancos_sangre',
    'sre442': 'uci',
    'sre443': 'uci_adultos',
    'sre444': 'uci_neonatal',
    'sre445': 'unidades_dentales'
}

In [54]:
egresos_resources.rename(columns=rename_dict, inplace=True)
egresos_resources.head(2)

,anio,lat_decimal,lon_decimal,personal_medico_nomina,personal_medico_acuerdo,contacto_directo_paciente_nomina,contacto_directo_paciente_acuerdo,medicos_generales_nomina,medicos_generales_acuerdo,especialistas_nomina,especialistas_acuerdo,ginecoobstetras_nomina,ginecoobstetras_acuerdo,pediatras_nomina,pediatras_acuerdo,cirujanos_nomina,cirujanos_acuerdo,internistas_nomina,internistas_acuerdo,anestesiologos_nomina,anestesiologos_acuerdo,otros_especialistas_nomina,otros_especialistas_acuerdo,odontologos_nomina,odontologos_acuerdo,residentes_nomina,residentes_acuerdo,pasantes_nomina,pasantes_acuerdo,medicos_otras_labores_nomina,medicos_otras_labores_acuerdo,personal_no_medico,diagnostico_medico,tratamiento_medico,personal_paramedico,auxiliares_enfermeria,enfermeras_generales,enfermeras_especializadas,pasantes_enfermeria,otras_enfermeras,otro_personal_paramedico,personal_administrativo,otro_personal,consultorios,consultorios_generales,consultorios_especialidad,camas_censables,camas_medicina_interna,camas_cirugia,camas_ginecoobstetricia,camas_pediatria,camas_otras,camas_no_censables,camas_cuidado_intensivo,camas_cuidado_intermedio,camas_no_censables_otras,lab_analisis_clinicos,lab_anatomia_patologica,salas_radiologia,equipos_rayos_x,area_radioterapia,equipos_radioterapia,quirofanos,salas_expulsion,incubadoras,cunas_rn,area_pediatria,area_urgencias,area_aislamiento,resonancia_magnetica,equipo_dialisis,hemodialisis,mamografia,ultrasonido,electrocardiografo,endoscopio,electroencefalografo,litotriptores,tac_scanner,bomba_cobalto,bancos_sangre,uci,uci_adultos,uci_neonatal,unidades_dentales,lat_trunc_res,lon_trunc_res,clues,edad_anios,sexo,servicio_troncal,tipo_derechohabiente,fecha_ingreso,fecha_egreso,diagnostico_principal_cie10,descripcion_diagnostico,clave_institucion,nombre_institucion,clave_entidad_egresos,nombre_entidad,clave_municipio_egresos,nombre_municipio,clave_localidad,nombre_localidad,nombre_unidad,codigo_postal,latitud,longitud,nivel_atencion,lat_trunc,lon_trunc
0,2024,32.641176,-115.475578,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,72,hombre,Cirugia General,pensionado,12/01/2024 0:00,19/01/2024 0:00,C61.X,Tumor maligno de la prostata,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4
1,2024,32.641176,-115.475578,40,653,31,645,25,98,6,526,2,59,1,71,1,84,1,30,0,58,1,224,0,1,0,14,0,6,9,8,1310,88,0,601,63,389,43,24,18,64,251,370,42,25,17,232,49,28,31,6,118,119,29,10,80,8,4,8,22,0,0,36,11,45,45,4,22,5,3,0,1,5,21,23,9,4,2,3,0,0,32,20,12,0,32.6,-115.4,BCIST000040,73,mujer,Cirugia General,madre,15/01/2024 0:00,22/01/2024 0:00,M17.9,Gonartrosis no especificada,IST,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,2,BAJA CALIFORNIA,2,MEXICALI,1,MEXICALI,"HG ""5 DE DICIEMBRE"", MEXICALI",21000.0,32.6438,-115.47638,SEGUNDO NIVEL,32.6,-115.4


## Export Output File

In [55]:
egresos_resources.to_csv('../data/egresos_resources.csv', index=False)